## Task 2 : Independent Implementation of RSA

In [59]:
import os
import time

def get_power_2_factors(n: int) -> (int, int):
    r = 0
    d = n
    while n > 0 and d % 2 == 0:
        d = d // 2
        r += 1
    return r, d


def miller_rabin_prime_test(n: int, k: int) -> bool:

    # Factor powers of 2 from n - 1 s.t. n - 1 = 2^r * d
    r, d = get_power_2_factors(n-1)

    for i in range(k):
        a = get_random_bits(n.bit_length())
        while a not in range(2, n-2+1):
            a = get_random_bits(n.bit_length())
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        n_1_found = False
        for j in range(r-1):
            x = pow(x, 2, n)
            if x == n - 1:
                n_1_found = True
                break
        if not n_1_found:
            return False
    return True


def get_random_bits(bit_length: int) -> int:
    return int.from_bytes(os.urandom((bit_length + 7) // 8), 'big')


def generate_prime_number(bit_length: int) -> int:

    # prime needs to be in range [2^(n-1), 2^n-1]
    low = pow(2, bit_length - 1)
    high = pow(2, bit_length) - 1

    while True:

        # Generate odd prime candidate in range
        candidate_prime = get_random_bits(bit_length)
        while candidate_prime not in range(low, high+1) or not candidate_prime % 2:
            candidate_prime = get_random_bits(bit_length)

        # with k rounds, miller rabin test gives false positive with probability (1/4)^k = 1/(2^2k)
        k = 64
        if miller_rabin_prime_test(candidate_prime, k):
            return candidate_prime


def extended_gcd(a, b):
    if not b:
        return 1, 0

    u, v = extended_gcd(b, a % b)
    return v, u - v * (a // b)


def calculate_private_key(e: int, p: int, q: int) -> int:
    u, _ = extended_gcd(e, (p-1)*(q-1))
    return u


def rsa_encrypt(plaintext, e, n):
    ciphertext = []
    for p in plaintext:
        p_ascii = ord(p)
        x = pow(p_ascii, e, n)
        ciphertext.append(x)
    return ciphertext




def rsa_decrypt(ciphertext, d, n):
    deciphertext = ""
    for c in ciphertext:
        x = pow(c, d, n)
        deciphertext+= chr(x)
    return deciphertext




### RSA implementation for predefined value of e,n,d

In [60]:
def rsa(rsa_key_size, plaintext, n, e, d):

    prime_number_bit_length = rsa_key_size // 2

    start_time = time.perf_counter()

    end_time = time.perf_counter()
    key_generation_time = end_time - start_time
    print("Key Generation Time:", key_generation_time, "seconds")


    start_time = time.perf_counter()

    cipher = rsa_encrypt(plaintext, e, n)

    end_time = time.perf_counter()
    encryption_time = end_time - start_time
    print("Encryption Time:", encryption_time, "seconds")

    print("Cipher Text:", cipher)

    start_time = time.perf_counter()

    decipher = rsa_decrypt(cipher, d, n)

    end_time = time.perf_counter()
    decryption_time = end_time - start_time
    print("Decryption Time:", decryption_time, "seconds")

    print("Deciphered Text:", decipher)


In [61]:
rsa_key_size=16
n=60491
e=7
d=17143

plaintext = "BUETCSEVSSUSTCSE"
rsa(rsa_key_size, plaintext, n, e, d)

Key Generation Time: 5.00003807246685e-07 seconds
Encryption Time: 1.7200014553964138e-05 seconds
Cipher Text: [53296, 32514, 13868, 59107, 55418, 47636, 13868, 21544, 47636, 47636, 32514, 47636, 59107, 55418, 47636, 13868]
Decryption Time: 3.709999145939946e-05 seconds
Deciphered Text: BUETCSEVSSUSTCSE


In [62]:
rsa_key_size=32
n=4292870399
e=11
d=1560996131

plaintext = "BUETCSEVSSUSTCSE"
rsa(rsa_key_size, plaintext, n, e, d)

Key Generation Time: 6.00004568696022e-07 seconds
Encryption Time: 2.0400038920342922e-05 seconds
Cipher Text: [1268954923, 3419348944, 3190642909, 1478590601, 1579240218, 2079935148, 3190642909, 1466721226, 2079935148, 2079935148, 3419348944, 2079935148, 1478590601, 1579240218, 2079935148, 3190642909]
Decryption Time: 0.00014809996355324984 seconds
Deciphered Text: BUETCSEVSSUSTCSE


In [63]:
rsa_key_size=64
n=18446743979220271189
e=3
d=12297829313753557747

plaintext = "BUETCSEVSSUSTCSE"
rsa(rsa_key_size, plaintext, n, e, d)

Key Generation Time: 4.00003045797348e-07 seconds
Encryption Time: 1.109996810555458e-05 seconds
Cipher Text: [287496, 614125, 328509, 592704, 300763, 571787, 328509, 636056, 571787, 571787, 614125, 571787, 592704, 300763, 571787, 328509]
Decryption Time: 0.0003766000154428184 seconds
Deciphered Text: BUETCSEVSSUSTCSE


In [64]:
rsa_key_size=96
n=79228162514229434696431832827
e=3
d=52818775009485914497652274427

plaintext = "BUETCSEVSSUSTCSE"
rsa(rsa_key_size, plaintext, n, e, d)

Key Generation Time: 3.00002284348011e-07 seconds
Encryption Time: 1.2499978765845299e-05 seconds
Cipher Text: [287496, 614125, 328509, 592704, 300763, 571787, 328509, 636056, 571787, 571787, 614125, 571787, 592704, 300763, 571787, 328509]
Decryption Time: 0.0005474999779835343 seconds
Deciphered Text: BUETCSEVSSUSTCSE


### RSA implementation with randomly generated value of p & q

In [65]:
def rsa(rsa_key_size, plaintext):

    prime_number_bit_length = rsa_key_size // 2

    start_time = time.perf_counter()

    # Generate prime numbers p and q
    p = generate_prime_number(prime_number_bit_length)
    q = generate_prime_number(prime_number_bit_length)

    end_time = time.perf_counter()
    key_generation_time = end_time - start_time
    print("Key Generation Time:", key_generation_time, "seconds")

    n = p * q
    e = 65537
    
    d = calculate_private_key(e,p,q)
    
    start_time = time.perf_counter()

    cipher = rsa_encrypt(plaintext, e, n)

    end_time = time.perf_counter()
    encryption_time = end_time - start_time
    print("Encryption Time:", encryption_time, "seconds")

    print("Cipher Text:", cipher)

    # Decrypt
    start_time = time.perf_counter()

    decipher = rsa_decrypt(cipher, d, n)

    end_time = time.perf_counter()
    decryption_time = end_time - start_time
    print("Decryption Time:", decryption_time, "seconds")

    print("Deciphered Text:", decipher)


In [66]:
plaintext = input('Please enter the plaintext : ')
#BUETCSEVSSUSTCSE
rsa_key_size = int(input("Please enter key size(16/32/64/96) : "))

rsa(rsa_key_size, plaintext)

Key Generation Time: 0.0006315999780781567 seconds
Encryption Time: 7.100001676008105e-05 seconds
Cipher Text: [1670895383, 921725888, 1108904878, 1391361755, 1335792962, 115770154, 1108904878, 2066953858, 115770154, 115770154, 921725888, 115770154, 1391361755, 1335792962, 115770154, 1108904878]
Decryption Time: 0.00020589999621734023 seconds
Deciphered Text: BUETCSEVSSUSTCSE
